In [12]:
import torch as tr
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.optim import SGD
class neural_network(nn.Module):
    def __init__(self, layers=[2, 100, 2], scale=.1, p=None, lr=None, lam=None):
        super().__init__()
        self.weights = nn.ParameterList([nn.Parameter(scale * tr.randn(m, n)) for m, n in zip(layers[:-1], layers[1:])])
        self.biases = nn.ParameterList([nn.Parameter(scale * tr.randn(n)) for n in layers[1:]])
        # self.weights = None
        # self.biases = None
        self.p = p
        self.lr = lr
        self.lam = lam
        self.train_mode = False

    def relu(self, X, W, b):
        return F.relu(X @ W + b)

    def softmax(self, X, W, b):
        return F.softmax(X @ W + b, dim=1)

    def forward(self, X):
        X = tr.tensor(X, dtype=tr.float)
        for i in range(len(self.weights) - 1):
            X = self.relu(X, self.weights[i], self.biases[i])
            if self.p is not None and self.train_mode:
                X = F.dropout(X, p=self.p)

        X = self.softmax(X, self.weights[-1], self.biases[-1])
        return X

    def predict(self, X):
        return self.forward(X).detach().numpy()

    def loss(self, ypred, ytrue):
        ypred = tr.clamp(ypred, 1e-9, 1 - 1e-9)  # To avoid log(0)
        return -tr.mean(tr.sum(ytrue * tr.log(ypred), dim=1))

    def fit(self, X, y, nsteps=1000, bs=100, plot=False):
        X, y = tr.tensor(X, dtype=tr.float), tr.tensor(y, dtype=tr.float)
        optimizer = SGD(self.parameters(), lr=self.lr, weight_decay=self.lam)

        I = tr.randperm(X.shape[0])
        n = int(np.floor(.9 * X.shape[0]))
        Xtrain, ytrain = X[I[:n]], y[I[:n]]
        Xval, yval = X[I[n:]], y[I[n:]]

        Ltrain, Lval, Aval = [], [], []
        for i in range(nsteps):
            optimizer.zero_grad()
            I = tr.randperm(Xtrain.shape[0])[:bs]
            self.train_mode = True
            output = self.loss(self.forward(Xtrain[I]), ytrain[I])
            self.train_mode = False
            Ltrain += [output.item()]
            output.backward()
            optimizer.step()

            outval = self.forward(Xval)
            Lval += [self.loss(outval, yval).item()]
            Aval += [np.array(outval.argmax(-1) == yval.argmax(-1)).mean()]

        if plot:
            plt.plot(range(nsteps), Ltrain, label='Training loss')
            plt.plot(range(nsteps), Lval, label='Validation loss')
            plt.plot(range(nsteps), Aval, label='Validation acc')
            plt.legend()
            plt.show()

# # Example for testing the neural_network class
# if __name__ == "__main__":
#     # Create dummy data for testing
#     X = np.random.randn(200, 2)
#     y = np.eye(2)[(X[:, 0] + X[:, 1] > 0).astype(int)]  # Simple linearly separable data

#     # Initialize and train the neural network
#     nn = neural_network(layers=[2, 100, 2], scale=.1, lr=0.01, lam=0.01, p=0.5)
#     nn.fit(X, y, nsteps=1000, bs=20, plot=True)

#     # Predict on new data
#     X_test = np.random.randn(50, 2)
#     y_pred = nn.predict(X_test)
#     print(y_pred)


In [17]:
import unittest

def test_neural_network(self):
        X = tr.tensor([[1, 1], [0, 0]], dtype=tr.float)
        y = tr.tensor([[0, 1], [1, 0]], dtype=tr.int)
        W = tr.tensor([[1, .2], [.5, 1]], dtype=tr.float)
        b = tr.tensor([-1, -1], dtype=tr.float)
        m = neural_network(layers=[2,2,2], p=0, lam=0, lr=.1)

        m.fit(X, y, nsteps=1, bs=1, plot=False)

        relu_out = tr.tensor([[.5,.2],[0, 0]])
        # self.assertTrue(np.allclose(m.relu(X, W, b), relu_out), msg='neural_network: Error. ReLU output not correct')

        softmax_out = np.array([[0.57444252, 0.42555748], [.5, .5]])
        # self.assertTrue(np.allclose(m.softmax(relu_out, W, b), softmax_out), msg='neural_network: Error. Softmax output not correct')

        m.weights = tr.nn.ParameterList([tr.nn.Parameter(W), tr.nn.Parameter(W)])
        m.biases = tr.nn.ParameterList([tr.nn.Parameter(b), tr.nn.Parameter(b)])
        loss_out = 0.7737512125142362
        out = m.forward(X)
        loss = m.loss(out, y).item()
        # self.assertTrue(np.isclose(loss_out, loss), msg='neural_network: Error. Loss output not correct')
        # self.assertTrue(np.allclose(out.detach().numpy(), softmax_out), msg='neural_network: Error. Network output not correct')
        
test_neural_network(unittest.TestCase)

C:\Users\Tue\AppData\Local\Temp\ipykernel_26276\1711889869.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X, y = tr.tensor(X, dtype=tr.float), tr.tensor(y, dtype=tr.float)
C:\Users\Tue\AppData\Local\Temp\ipykernel_26276\1711889869.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = tr.tensor(X, dtype=tr.float)
